In [29]:
# 인터넷 쇼핑몰의 분야별 베스트셀러 상품 크롤러
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import random
import os
import urllib.request
import urllib.parse

# Step 2. 사용자에게 카테고리 메뉴를 보여주고 정보를 입력 받습니다.
print("=" *80)
print("     아마존 닷컴의 분야별 Best Seller 상품 정보 추출하기")
print("     확인하고자하는 item 선택후 Top 50 제품의 제품명과 가격, 평점 확인")
print("     .csv file 형태로 요청 폴더에 저장")
print("=" *80)
query_txt='아마존닷컴'
query_url='https://www.amazon.com/bestsellers?ld=NSGoogle'

res=urllib.request.urlopen(query_url)
html = res.read()
soup = BeautifulSoup(html, 'html.parser')
result = soup.find('ul', id='zg_browseRoot').find('ul')
slist = result.find_all('li')

sec_text_list=[]             # 첫번째항목 두번째항목...
sec_list=[]                  # 1.항목 2.항목...
count=1

for li in slist:
    sec_cnt=str(count)       # 카운터 스트링숫자
    sec_text = li.get_text() # 항목의 텍스트
    sec_text_list.append(sec_text) # 첫번째항목 두번째항목...
    sec_list.append(sec_cnt+"."+sec_text) # 1.항목 2.항목...
    count+=1
sec_input=[]

df_sec_text_list = pd.DataFrame(sec_text_list)
df_sec_text_list.columns = ['items']
print(df_sec_text_list)

sec_input.append('\n\n'+'*.위 List에서 수집할 Item의 번호를  선택하세요: ')
sec = input(str(''.join(sec_input)))
sec = int(sec)
s_no = sec+2

s_item = df_sec_text_list['items'].iloc[sec]
# print (s_item) 검색할 아이템명 출력

f_dir = input(" *.파일을 저장할 폴더명만 쓰세요(예:C:\\Users\\miero\\Documents\\data\\):")
cnt = 50

print("\n")

for c in range(1,cnt+1):
    if sec==str(c):
        sec_name=sec_text_list[c-1]

print("=" *80)
print ('     요청하신 '+s_item+'의 Top 50을 확인하여 ')
print ('     '+f_dir+'에 CSV file로 저장됩니다.')
print("=" *80)

# 파일이 저정될 directory 설정과 폴더 생생. 파일명 지정(생성)
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
os.makedirs(f_dir+s+'-'+query_txt+'-'+s_item)
os.chdir(f_dir+s+'-'+query_txt+'-'+s_item)
ff_dir = f_dir+s+'-'+query_txt+'-'+s_item

ff_name = f_dir+s+'-'+query_txt+'-'+s_item+'\\'+s+'-'+query_txt+'-'+s_item+'.txt'
fc_name = f_dir+s+'-'+query_txt+'-'+s_item+'\\'+s+'-'+query_txt+'-'+s_item+'.csv'
s_time = time.time( )

path = "D:/00_Limhs/py_data/drv/chromedriver.exe" # 학원
# path = "D:/py_WS/drv/chromedriver.exe" # 집

driver = webdriver.Chrome(path)
driver.get(query_url)
time.sleep(2)
driver.refresh() # link로 접근하면 바로 접근이 되지 않아 새로고침을 실행
time.sleep(2)

a = "/html/body/div[1]/div[2]/div[2]/div/div/div/div[2]/div/div[2]/div/ul/ul/li[{s_no}]/a".format(s_no=s_no)
driver.find_element_by_xpath(a).click()

#Step 4. 화면을 스크롤해서 아래로 이동한 후 요청된 데이터를 수집합니다.
def scroll_down(driver):
    driver.execute_script("window.scrollBy(0,9300);")
    time.sleep(1)
scroll_down(driver)

# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만듭니다
# bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
reple_result = soup.select('#zg-center-div > #zg-ordered-list')
slist = reple_result[0].find_all('li')

ranking2=[]
title3=[]
price2=[]
score2=[]
sat_count2=[]
store2=[]
url_list = []

count = 0
    
for li in slist:
    f = open(ff_name, 'a',encoding='UTF-8')
    f.write("-----------------------------------------------------"+"\n")

    # 판매순위
    print("-" *70)
    try :
        ranking = li.find('span',class_='zg-badge-text').get_text().replace("#","")
    except AttributeError :
        ranking = ''
        print(ranking.replace("#",""))
    else :
        print("-.판매순위:",ranking)
        f.write('-.판매순위:'+ ranking + "\n")

    #제품 설명 
    try :
        title1 = li.find('div',class_='p13n-sc-truncated').get_text().replace("\n","")
    except AttributeError :
        title1 = ''
        print(title1.replace("\n",""))
        f.write('-.제품소개:'+ title1 + "\n")
    else :
        title2=title1.translate(bmp_map).replace("\n","") 
        print("-.제품소개:", title2.replace("\n",""))
        count += 1
        f.write('-.제품소개:'+ title2 + "\n")
    # 가격
    try :
        price = li.find('span','p13n-sc-price').get_text().replace("\n","")
    except AttributeError :
        price = ''
        print("-.가격:", price.replace("\n",""))
        f.write('-.가격:'+ price + "\n")
    # 상품평수
    try :
        sat_count = li.find('a','a-size-small a-link-normal').get_text().replace(",","")
    except (IndexError , AttributeError) :
        sat_count='0'
        print('-.상품평 수: ',sat_count)
        f.write('-.상품평 수:'+ sat_count + "\n")
    else :
        print('-.상품평 수:',sat_count)
        f.write('-.상품평 수:'+ sat_count + "\n")

    # URL link
    try :
        aTag = li.find('a' , class_ = 'a-link-normal')
        url1 = aTag.attrs['href']
        url = ("https://www.amazon.com" + url1)

    except AttributeError :
        url_list=''
        print('-.url_link: ',url_list)
        f.write('-.url_link:'+ url_list + "\n")

    #상품 별점 구하기

    try :
        score = li.find('span','a-icon-alt').get_text()
    except AttributeError :
        score=' '
        print('-.평점:',score)
        f.write('-.평점:'+ score + "\n")
        print("-" *70)
        f.close( )
        time.sleep(0.3)
    ranking2.append(ranking)
    title3.append(title2.replace("\n",""))
    price2.append(price.replace("\n",""))
    url_list.append(url)

    try :   
        sat_count2.append(sat_count)
    except IndexError :
        sat_count2.append(0)

    score2.append(score)
    if count == cnt :
        break
        
#Step 5. 검색 결과를 다양한 형태로 저장하기
amazon_best_seller = pd.DataFrame()
amazon_best_seller['판매순위']=ranking2
amazon_best_seller['제품소개']=pd.Series(title3)
amazon_best_seller['판매가격']=pd.Series(price2)
amazon_best_seller['상품평 갯수']=pd.Series(sat_count2)
amazon_best_seller['상품평점']=pd.Series(score2)
amazon_best_seller['URL']=pd.Series(url_list)

# csv 형태로 저장하기
amazon_best_seller.to_csv(fc_name,encoding="utf-8-sig",index=True)

# txt 파일에 크롤링 요약 정보 저장하기
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f

# Step 6. 요약 정보를 출력하기

e_time = time.time( )
t_time = e_time - s_time

sys.stdout = orig_stdout
f.close( )

print("\n") 
print("=" *80)
print("1.요청된 정보 수집이 완료 되었습니다." )
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: %s 폴더에 저장되었습니다." %f_dir)
print("=" *80)
driver.close()

     아마존 닷컴의 분야별 Best Seller 상품 정보 추출하기
     확인하고자하는 item 선택후 Top 50 제품의 제품명과 가격, 평점 확인
     .csv file 형태로 요청 폴더에 저장
                            items
0    Amazon Devices & Accessories
1                Amazon Launchpad
2                      Appliances
3                    Apps & Games
4           Arts, Crafts & Sewing
5       Audible Books & Originals
6                      Automotive
7                            Baby
8          Beauty & Personal Care
9                           Books
10                    CDs & Vinyl
11        Camera & Photo Products
12      Cell Phones & Accessories
13      Clothing, Shoes & Jewelry
14         Collectible Currencies
15        Computers & Accessories
16  Digital Educational Resources
17                  Digital Music
18                    Electronics
19     Entertainment Collectibles
20                     Gift Cards
21         Grocery & Gourmet Food
22              Handmade Products
23             Health & Household
24                 Home & Kitchen

In [28]:
print(url)

https://www.amazon.com//Raceway-Management-Channel-Paintable-Concealer/dp/B07GPFDL1K/ref=zg_bs_electronics_50?_encoding=UTF8&psc=1&refRID=G31Y352559BSG4MS0MDQ


In [2]:
import time
time.localtime()

time.struct_time(tm_year=2021, tm_mon=10, tm_mday=22, tm_hour=9, tm_min=58, tm_sec=52, tm_wday=4, tm_yday=295, tm_isdst=0)